# 🚀 IBM watsonx.ai + CrewAI Workshop

---

## Workshop Overview

Welcome to this hands-on workshop where you'll learn to build intelligent multi-agent systems using **IBM watsonx.ai** and **CrewAI**!

### What You'll Learn

By the end of this workshop, you will be able to:

1. ✅ Connect to IBM watsonx.ai from Python
2. ✅ Create custom LLM wrappers for CrewAI
3. ✅ Design and implement multi-agent workflows
4. ✅ Build intelligent AI agents with specialized roles
5. ✅ Orchestrate complex tasks using agent collaboration

### Prerequisites

- Python 3.10 or higher
- IBM Cloud account with watsonx.ai access
- Basic Python programming knowledge
- Familiarity with AI/LLM concepts (helpful but not required)

### What You'll Need

Before starting, make sure you have:

- 🔑 **IBM Cloud API Key** for watsonx.ai
- 🌐 **Service URL** for your region (e.g., `https://us-south.ml.cloud.ibm.com`)
- 📁 **Project ID** from your watsonx.ai project

> 💡 **Tip**: If you don't have these yet, visit [IBM Cloud](https://cloud.ibm.com) to set up your watsonx.ai instance.

---

### Workshop Structure

| Section | Topic | Duration |
|---------|-------|----------|
| 0 | Environment Setup | 5 min |
| 1 | watsonx.ai Configuration | 5 min |
| 2 | Quick Test & Validation | 5 min |
| 3 | Building Custom LLM Wrapper | 15 min |
| 4 | Creating Your First Agent | 10 min |
| 5 | Multi-Agent Workflow | 20 min |
| 6 | Real-World Use Case | 20 min |
| 7 | Exercises & Challenges | 15 min |

---

**Let's get started! 🎯**

---

## 📦 Section 0: Environment Setup

First, we'll install all required packages and configure the environment.

### What We're Installing

- **crewai[tools]**: Multi-agent orchestration framework
- **langchain-ibm**: IBM watsonx.ai integration for LangChain
- **ibm-watsonx-ai**: Official IBM watsonx SDK
- **python-dotenv**: Environment variable management

> ⏱️ This may take 1-2 minutes to complete.

In [ ]:
# Install required packages
!pip install -q -U "crewai[tools]" langchain-ibm ibm-watsonx-ai python-dotenv

print("✅ All packages installed successfully!")

### Disable Telemetry

We'll disable CrewAI's telemetry to avoid interruptions during the workshop.
This prevents timeout prompts and keeps our notebook running smoothly.

In [ ]:
import os

# Disable CrewAI telemetry and tracing
os.environ["CREWAI_DISABLE_TELEMETRY"] = "true"
os.environ["CREWAI_TELEMETRY"] = "false"
os.environ["CREWAI_TRACING_ENABLED"] = "false"
os.environ["OTEL_SDK_DISABLED"] = "true"

print("✅ Telemetry disabled")

### Version Check

Let's verify that all packages are installed correctly.

In [ ]:
import sys
import platform

print("="*50)
print("ENVIRONMENT INFORMATION")
print("="*50)
print(f"Python Version: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print()

# Check installed packages
packages = {
    "crewai": "CrewAI",
    "langchain_ibm": "LangChain-IBM",
    "ibm_watsonx_ai": "IBM watsonx.ai SDK"
}

for module_name, display_name in packages.items():
    try:
        module = __import__(module_name)
        version = getattr(module, "__version__", "unknown")
        print(f"✅ {display_name}: {version}")
    except ImportError as e:
        print(f"❌ {display_name}: Not installed - {e}")

print("="*50)
print("Environment setup complete!")
print("="*50)

---

## 🔐 Section 1: Configure IBM watsonx.ai Credentials

### Understanding watsonx.ai Authentication

IBM watsonx.ai uses **API key-based authentication**. You'll need three pieces of information:

#### 1. IBM Cloud API Key 🔑
- Used to authenticate your requests
- Can be created in IBM Cloud IAM
- Keep this secure and never commit to version control

#### 2. Service URL 🌐
Choose the URL based on your region:

| Region | URL |
|--------|-----|
| Dallas (US South) | `https://us-south.ml.cloud.ibm.com` |
| Frankfurt (EU DE) | `https://eu-de.ml.cloud.ibm.com` |
| London (EU GB) | `https://eu-gb.ml.cloud.ibm.com` |
| Tokyo (JP TOK) | `https://jp-tok.ml.cloud.ibm.com` |
| Sydney (AU SYD) | `https://au-syd.ml.cloud.ibm.com` |

#### 3. Project ID 📁
- Found in your watsonx.ai project settings
- Identifies which project's resources to use

### Security Best Practices 🛡️

- ✅ Use environment variables for credentials
- ✅ Never hardcode API keys in code
- ✅ Use `.env` files for local development (don't commit them!)
- ✅ Rotate API keys regularly
- ✅ Use least-privilege access principles

---

Run the cell below to enter your credentials securely:

In [ ]:
from getpass import getpass

print("🔐 IBM watsonx.ai Credentials Setup")
print("="*50)
print("Your credentials will be stored securely in environment variables.")
print("They will NOT be displayed or saved to disk.\n")

# Securely input API key (masked)
WATSONX_API_KEY = getpass("Enter your IBM Cloud API Key: ")

# Input Service URL
print("\nCommon URLs:")
print("  US: https://us-south.ml.cloud.ibm.com")
print("  EU: https://eu-de.ml.cloud.ibm.com")
WATSONX_URL = input("\nEnter your watsonx.ai URL: ").strip()

# Input Project ID
WATSONX_PROJECT_ID = input("Enter your Project ID: ").strip()

# Store in environment variables
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY
os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

print("\n✅ Credentials configured successfully!")
print(f"   Region URL: {WATSONX_URL}")
print(f"   Project ID: {WATSONX_PROJECT_ID[:8]}...")

---

## ✅ Section 2: Quick Test & Validation

Before building our multi-agent system, let's verify that everything works correctly.

### What This Test Does

1. Creates a connection to watsonx.ai
2. Sends a simple chat message
3. Verifies the response

### About the Model

We're using **IBM Granite 3 8B Instruct** - a powerful, efficient model suitable for:
- Conversational AI
- Technical assistance
- Code generation
- Multi-agent workflows

> 💡 **Note**: You can change the model ID to any model available in your watsonx.ai instance.

In [ ]:
from langchain_ibm import ChatWatsonx

# Model configuration
WATSONX_MODEL_ID = "ibm/granite-3-8b-instruct"

print(f"🤖 Testing connection with model: {WATSONX_MODEL_ID}")
print("="*70)

# Configure model parameters
parameters = {
    "temperature": 0.3,      # Lower = more deterministic
    "max_tokens": 256,       # Maximum response length
    "top_p": 0.95,           # Nucleus sampling threshold
}

# Initialize the chat model
chat = ChatWatsonx(
    model_id=WATSONX_MODEL_ID,
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)

# Test message
messages = [
    ("system", "You are a helpful AI assistant specializing in multi-agent systems."),
    ("human", "Explain what CrewAI is in exactly 3 bullet points."),
]

print("⏳ Sending test message to watsonx.ai...\n")

try:
    response = chat.invoke(messages)
    print("✅ CONNECTION SUCCESSFUL!")
    print("="*70)
    print("Response:\n")
    print(response.content)
    print("="*70)
    print("\n✨ Your watsonx.ai connection is working perfectly!")
except Exception as e:
    print("❌ CONNECTION FAILED")
    print(f"Error: {e}")
    print("\nPlease check:")
    print("  1. Your API key is correct")
    print("  2. Your URL matches your region")
    print("  3. Your Project ID is valid")
    print("  4. The model is available in your instance")

---

## 🔧 Section 3: Building a Custom LLM Wrapper for CrewAI

### Understanding the Integration

CrewAI needs language models to follow a specific interface. While it has built-in support for many providers, watsonx.ai requires a custom wrapper.

### Architecture Overview

```
┌─────────────────┐
│   CrewAI Agent  │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  WatsonxLLM     │ ◄── Our Custom Wrapper
│  (BaseLLM)      │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  ChatWatsonx    │ ◄── LangChain Integration
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  watsonx.ai API │
└─────────────────┘
```

### Key Components

1. **BaseLLM**: CrewAI's abstract base class for language models
2. **call()**: Main method that CrewAI uses to get completions
3. **ChatWatsonx**: Underlying connection to watsonx.ai

### Features We're Implementing

- ✅ Simple text-based chat
- ✅ Message history handling
- ✅ Configurable parameters (temperature, max_tokens)
- ✅ Error handling
- ⬜ Tool/function calling (future extension)

---

Let's build the wrapper:

In [ ]:
from typing import Any, Dict, List, Optional, Union
from crewai import BaseLLM
from langchain_ibm import ChatWatsonx


class WatsonxCrewAILLM(BaseLLM):
    """
    Custom LLM wrapper for using IBM watsonx.ai with CrewAI.
    
    This class bridges CrewAI's BaseLLM interface with IBM's ChatWatsonx,
    enabling watsonx.ai models to power CrewAI agents.
    
    Parameters:
        model_id (str): The watsonx.ai model identifier (e.g., 'ibm/granite-3-8b-instruct')
        url (str): The watsonx.ai service URL for your region
        project_id (str): Your watsonx.ai project ID
        api_key (str, optional): IBM Cloud API key. Reads from environment if not provided.
        temperature (float): Controls randomness (0.0 = deterministic, 1.0 = creative)
        max_tokens (int): Maximum number of tokens in the response
        top_p (float): Nucleus sampling threshold (0.0-1.0)
    
    Example:
        >>> llm = WatsonxCrewAILLM(
        ...     model_id="ibm/granite-3-8b-instruct",
        ...     url="https://us-south.ml.cloud.ibm.com",
        ...     project_id="your-project-id",
        ...     temperature=0.7
        ... )
    """
    
    def __init__(
        self,
        model_id: str,
        url: str,
        project_id: str,
        api_key: Optional[str] = None,
        temperature: float = 0.3,
        max_tokens: int = 512,
        top_p: float = 1.0,
    ) -> None:
        # Initialize parent BaseLLM class
        super().__init__(model=model_id, temperature=temperature)
        
        # Get API key from environment if not provided
        if api_key is None:
            api_key = os.getenv("WATSONX_APIKEY") or os.getenv("WATSONX_API_KEY")
        
        if not api_key:
            raise ValueError(
                "No IBM watsonx API key found. "
                "Set WATSONX_APIKEY/WATSONX_API_KEY environment variable "
                "or pass api_key parameter."
            )
        
        # Ensure environment variables are set for the SDK
        os.environ.setdefault("WATSONX_APIKEY", api_key)
        os.environ.setdefault("WATSONX_API_KEY", api_key)
        
        # Store configuration
        self.url = url
        self.project_id = project_id
        self.model_id = model_id
        
        # Initialize the underlying ChatWatsonx instance
        self._chat = ChatWatsonx(
            model_id=model_id,
            url=url,
            project_id=project_id,
            params={
                "temperature": temperature,
                "max_tokens": max_tokens,
                "top_p": top_p,
            },
        )
    
    def call(
        self,
        messages: Union[str, List[Dict[str, str]]],
        tools: Optional[List[dict]] = None,
        callbacks: Optional[List[Any]] = None,
        available_functions: Optional[Dict[str, Any]] = None,
        **kwargs
    ) -> str:
        """
        Main method called by CrewAI to get LLM completions.
        
        Args:
            messages: Either a string prompt or list of message dicts
            tools: Optional tool definitions (not used in this basic implementation)
            callbacks: Optional callbacks for monitoring
            available_functions: Optional function mappings
            **kwargs: Additional parameters passed by CrewAI
        
        Returns:
            str: The model's response text
        """
        # Handle different message formats
        if isinstance(messages, str):
            # Simple string prompt
            chat_input = messages
        else:
            # List of message dictionaries
            processed: List[tuple] = []
            for m in messages:
                role = m.get("role", "user")
                content = m.get("content", "")
                if content:
                    processed.append((role, content))
            
            chat_input = processed if processed else ""
        
        # Call watsonx.ai
        result = self._chat.invoke(chat_input)
        
        # Extract text content from response
        return getattr(result, "content", str(result))
    
    def supports_function_calling(self) -> bool:
        """
        Indicates whether this LLM supports tool/function calling.
        
        Returns:
            bool: False for this basic implementation
        """
        return False
    
    def get_context_window_size(self) -> int:
        """
        Returns the approximate context window size in tokens.
        
        Returns:
            int: Context window size (adjust based on your model)
        """
        return 8192


print("✅ WatsonxCrewAILLM class defined successfully!")
print("\nThis custom wrapper allows watsonx.ai models to power CrewAI agents.")

### Test the Custom Wrapper

Let's verify our wrapper works correctly before using it with agents.

In [ ]:
# Create an instance of our custom LLM
watsonx_llm = WatsonxCrewAILLM(
    model_id=WATSONX_MODEL_ID,
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    temperature=0.3,
    max_tokens=256,
)

print("🧪 Testing custom LLM wrapper...")
print("="*70)

# Test with a simple prompt
test_prompt = "Say hello from watsonx.ai and mention you're ready to power AI agents!"
print(f"Prompt: {test_prompt}\n")

try:
    response = watsonx_llm.call(test_prompt)
    print("✅ Wrapper Test Successful!")
    print("="*70)
    print("Response:\n")
    print(response)
    print("="*70)
    print("\n🎉 Your custom LLM wrapper is ready for CrewAI!")
except Exception as e:
    print(f"❌ Test failed: {e}")

---

## 🤖 Section 4: Creating Your First AI Agent

### What is a CrewAI Agent?

An **Agent** in CrewAI is an autonomous AI entity with:
- A specific **role** (its function in the team)
- A clear **goal** (what it's trying to achieve)
- A **backstory** (context that shapes its behavior)
- An **LLM** (the brain that powers it)

### Agent Design Principles

1. **Single Responsibility**: Each agent should have one clear purpose
2. **Clear Goals**: Goals should be specific and measurable
3. **Rich Backstory**: Provides context for better decision-making
4. **Appropriate Tools**: Agents work best with the right tools

### Our First Agent: The Research Specialist

Let's create a research agent that specializes in gathering and organizing information.

In [ ]:
from crewai import Agent

# Create a research agent
research_agent = Agent(
    role="Senior AI Research Specialist",
    
    goal=(
        "Conduct thorough research on technical topics and produce "
        "well-structured, accurate documentation that serves as a "
        "foundation for content creation."
    ),
    
    backstory=(
        "You are an experienced AI researcher with a PhD in Computer Science "
        "and 10 years of industry experience. You excel at breaking down "
        "complex technical concepts into clear, organized notes. Your research "
        "is always thorough, accurate, and well-cited. You have a talent for "
        "identifying the most important information and presenting it in a way "
        "that others can easily understand and build upon."
    ),
    
    llm=watsonx_llm,
    verbose=True,  # Show detailed agent reasoning
    allow_delegation=False,  # This agent works independently
)

print("✅ Research Agent Created!")
print("="*70)
print(f"Role: {research_agent.role}")
print(f"Goal: {research_agent.goal}")
print("="*70)
print("\n🎯 This agent is powered by watsonx.ai and ready to research!")

### Quick Agent Test

Let's give our agent a simple task to verify it works correctly.

In [ ]:
from crewai import Task

# Create a simple test task
test_task = Task(
    description=(
        "Research the concept of 'Agentic AI' and provide: \n"
        "1. A clear definition\n"
        "2. Three key characteristics\n"
        "3. Two real-world use cases"
    ),
    expected_output="A concise research summary in bullet-point format",
    agent=research_agent,
)

print("🔬 Testing research agent...\n")
print("="*70)

# Execute the task
result = test_task.execute()

print("\n" + "="*70)
print("RESEARCH RESULTS")
print("="*70)
print(result)
print("="*70)
print("\n✅ Agent test completed successfully!")

---

## 👥 Section 5: Building a Multi-Agent Workflow

### The Power of Agent Collaboration

Multi-agent systems excel at complex tasks by:
- **Dividing work** among specialized agents
- **Building on each other's outputs**
- **Iterating** towards better results
- **Providing checks and balances**

### Our Content Creation Crew

We'll build a three-agent system that works together to create high-quality content:

```
┌──────────────┐
│  Researcher  │ ─── Gathers & organizes information
└──────┬───────┘
       │
       ▼
┌──────────────┐
│   Writer     │ ─── Creates engaging content
└──────┬───────┘
       │
       ▼
┌──────────────┐
│   Editor     │ ─── Polishes & improves
└──────────────┘
```

### Benefits of This Approach

1. **Quality**: Multiple perspectives improve output
2. **Specialization**: Each agent focuses on what it does best
3. **Consistency**: Structured workflow ensures reliable results
4. **Scalability**: Easy to add more agents for complex tasks

### Step 1: Define All Agents

Let's create our complete team of specialized agents.

In [ ]:
from crewai import Agent

# Agent 1: Researcher
researcher = Agent(
    role="Senior AI Research Specialist",
    
    goal=(
        "Conduct comprehensive research on technical topics and produce "
        "well-organized, accurate documentation."
    ),
    
    backstory=(
        "You are an experienced AI researcher with deep expertise in enterprise "
        "AI, multi-agent systems, and IBM technologies. You excel at gathering "
        "comprehensive information, identifying key concepts, and organizing them "
        "into clear, structured notes. Your research serves as the foundation for "
        "high-quality content creation."
    ),
    
    llm=watsonx_llm,
    verbose=True,
)

# Agent 2: Technical Writer
writer = Agent(
    role="Senior Technical Writer",
    
    goal=(
        "Transform research notes into engaging, practical tutorial content "
        "that technical audiences can easily understand and apply."
    ),
    
    backstory=(
        "You are a skilled technical writer with 8 years of experience creating "
        "developer documentation and tutorials. You have a gift for explaining "
        "complex technical concepts in clear, accessible language. You understand "
        "how to structure content for maximum comprehension and engagement, and you "
        "always include practical examples that readers can use immediately."
    ),
    
    llm=watsonx_llm,
    verbose=True,
)

# Agent 3: Editor
editor = Agent(
    role="Content Quality Editor",
    
    goal=(
        "Review and enhance content for clarity, accuracy, structure, and "
        "pedagogical effectiveness, ensuring it's workshop-ready."
    ),
    
    backstory=(
        "You are a meticulous editor with a background in technical education. "
        "You have taught hundreds of workshops and know exactly what makes content "
        "effective for learning. You focus on improving clarity, fixing errors, "
        "enhancing structure, and ensuring content is accessible to the target audience. "
        "You're especially vigilant about security best practices and practical usability."
    ),
    
    llm=watsonx_llm,
    verbose=True,
)

print("✅ All agents created successfully!")
print("="*70)
print(f"1. {researcher.role}")
print(f"2. {writer.role}")
print(f"3. {editor.role}")
print("="*70)
print("\n🎭 Your multi-agent crew is assembled and ready!")

### Step 2: Define Tasks and Dependencies

Tasks define what each agent should do. We'll create a pipeline where each task builds on the previous one.

In [ ]:
from crewai import Task

# Define the topic (you can change this!)
WORKSHOP_TOPIC = "Building Intelligent Multi-Agent Systems with IBM watsonx.ai and CrewAI"

print(f"📚 Workshop Topic: {WORKSHOP_TOPIC}\n")

# Task 1: Research
research_task = Task(
    description=(
        f"Research the topic: '{WORKSHOP_TOPIC}'\n\n"
        "Your research should include:\n"
        "1. Overview of IBM watsonx.ai platform and its key capabilities\n"
        "2. Introduction to CrewAI framework and its use cases\n"
        "3. Benefits of combining watsonx.ai with multi-agent systems\n"
        "4. 8-10 practical use cases for watsonx-powered agents\n"
        "5. Architecture patterns and best practices\n"
        "6. Security considerations and cost optimization tips\n\n"
        "Format: Well-structured markdown with clear sections and bullet points."
    ),
    
    expected_output=(
        "A comprehensive research document (800-1000 words) in markdown format "
        "with sections for: watsonx.ai overview, CrewAI overview, use cases, "
        "architecture patterns, and best practices."
    ),
    
    agent=researcher,
)

# Task 2: Writing
writing_task = Task(
    description=(
        "Using the research notes, write a tutorial-style workshop article.\n\n"
        "The article should:\n"
        "1. Start with an engaging introduction\n"
        "2. Explain the architecture and how components work together\n"
        "3. Provide a step-by-step guide for building a simple agent\n"
        "4. Include practical tips and common pitfalls\n"
        "5. End with next steps and extension ideas\n\n"
        "Target: 1000-1500 words, easy to follow, with code examples if relevant."
    ),
    
    expected_output=(
        "A complete tutorial article in markdown format with clear headings, "
        "short paragraphs, numbered steps, and practical examples. Should be "
        "engaging and educational."
    ),
    
    agent=writer,
    context=[research_task],  # Uses output from research_task
)

# Task 3: Editing
editing_task = Task(
    description=(
        "Polish the draft article for a professional workshop audience.\n\n"
        "Focus on:\n"
        "1. Improving clarity and flow\n"
        "2. Fixing any technical inaccuracies\n"
        "3. Enhancing structure and readability\n"
        "4. Making security best practices explicit\n"
        "5. Ensuring code examples are complete and well-explained\n"
        "6. Adding helpful callouts or tips where appropriate\n\n"
        "The final output should be workshop-ready and professional."
    ),
    
    expected_output=(
        "A polished, professional tutorial article in markdown format, ready to be "
        "used in a live workshop or training session. Should be error-free, well-structured, "
        "and highly educational."
    ),
    
    agent=editor,
    context=[writing_task],  # Uses output from writing_task
)

print("✅ All tasks defined successfully!")
print("="*70)
print("Task Pipeline:")
print("  1. Research → Gather comprehensive information")
print("  2. Writing → Create tutorial content")
print("  3. Editing → Polish and finalize")
print("="*70)

### Step 3: Create and Execute the Crew

Now we'll assemble our agents and tasks into a crew and execute the workflow.

> ⏱️ **Note**: This may take 2-5 minutes to complete depending on the complexity of the topic.

In [ ]:
from crewai import Crew, Process

# Assemble the crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # Tasks run one after another
    memory=False,                # Disable memory system
    embedder=None,               # No vector database
    verbose=True,                # Show detailed execution logs
)

print("🚀 Starting content creation workflow...")
print("="*70)
print("This will take a few minutes. Please be patient!")
print("="*70)
print()

# Execute the crew
result = content_crew.kickoff()

# Extract final content
final_article = getattr(result, "raw", str(result))

print("\n\n" + "="*70)
print("🎉 CONTENT CREATION COMPLETED!")
print("="*70)
print("\n📄 Final Workshop Article:\n")
print(final_article)
print("\n" + "="*70)

### Save the Output

Let's save the generated article to a file.

In [ ]:
# Save to file
output_filename = "workshop_article.md"

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(final_article)

print(f"✅ Article saved to: {output_filename}")
print(f"📊 Length: {len(final_article)} characters")
print(f"📊 Words: ~{len(final_article.split())} words")

---

## 💼 Section 6: Real-World Use Case - Market Research Assistant

Let's build a practical multi-agent system for market research and competitive analysis.

### Scenario

You're a product manager who needs to:
1. Research competitors in a specific market
2. Analyze their strengths and weaknesses
3. Generate strategic recommendations

### Our Approach

We'll create a specialized crew:
- **Market Researcher**: Gathers competitive intelligence
- **Data Analyst**: Analyzes findings and identifies patterns
- **Strategy Consultant**: Provides actionable recommendations

In [ ]:
# Define market research agents
market_researcher = Agent(
    role="Senior Market Research Analyst",
    goal="Gather comprehensive competitive intelligence and market insights",
    backstory=(
        "You are an experienced market researcher with 12 years in the tech industry. "
        "You excel at identifying market trends, analyzing competitors, and gathering "
        "actionable intelligence that drives business decisions."
    ),
    llm=watsonx_llm,
    verbose=True,
)

data_analyst = Agent(
    role="Senior Data Analyst",
    goal="Analyze market data and identify strategic patterns and opportunities",
    backstory=(
        "You are a data analyst with expertise in competitive analysis and market positioning. "
        "You can identify patterns in complex data and translate findings into clear insights "
        "that business leaders can act on."
    ),
    llm=watsonx_llm,
    verbose=True,
)

strategy_consultant = Agent(
    role="Business Strategy Consultant",
    goal="Provide actionable strategic recommendations based on market analysis",
    backstory=(
        "You are a strategy consultant who has advised Fortune 500 companies. "
        "You excel at turning market insights into concrete action plans with clear "
        "priorities and measurable outcomes."
    ),
    llm=watsonx_llm,
    verbose=True,
)

# Define the market and target
MARKET_FOCUS = "Enterprise AI platforms for multi-agent workflows"

# Define tasks
market_research_task = Task(
    description=(
        f"Research the market: '{MARKET_FOCUS}'\n\n"
        "Provide:\n"
        "1. Overview of the market landscape\n"
        "2. Key players and their offerings\n"
        "3. Market size and growth trends\n"
        "4. Customer pain points and needs\n"
        "5. Emerging trends and technologies"
    ),
    expected_output="Structured market research report with clear sections",
    agent=market_researcher,
)

analysis_task = Task(
    description=(
        "Analyze the market research and identify:\n"
        "1. Competitive strengths and weaknesses\n"
        "2. Market gaps and opportunities\n"
        "3. Differentiation factors\n"
        "4. Risk factors\n"
        "5. Success patterns among leaders"
    ),
    expected_output="Analytical report with SWOT-style insights",
    agent=data_analyst,
    context=[market_research_task],
)

strategy_task = Task(
    description=(
        "Based on the research and analysis, provide:\n"
        "1. Top 5 strategic recommendations\n"
        "2. Priority actions for the next quarter\n"
        "3. Resource requirements and timeline\n"
        "4. Success metrics and KPIs\n"
        "5. Risk mitigation strategies"
    ),
    expected_output="Executive strategy document with actionable recommendations",
    agent=strategy_consultant,
    context=[analysis_task],
)

# Create and run the market research crew
market_crew = Crew(
    agents=[market_researcher, data_analyst, strategy_consultant],
    tasks=[market_research_task, analysis_task, strategy_task],
    process=Process.sequential,
    memory=False,
    embedder=None,
    verbose=True,
)

print(f"🔍 Analyzing market: {MARKET_FOCUS}")
print("="*70)
print("Starting market research workflow...\n")

market_result = market_crew.kickoff()
strategy_report = getattr(market_result, "raw", str(market_result))

print("\n" + "="*70)
print("📊 MARKET RESEARCH COMPLETE")
print("="*70)
print("\nStrategy Report:\n")
print(strategy_report)
print("\n" + "="*70)

---

## 🎯 Section 7: Exercises & Challenges

Now it's your turn to practice! Try these exercises to deepen your understanding.

### Exercise 1: Modify the Topic (Easy)

**Goal**: Change the workshop topic and run the content creation crew again.

**Instructions**:
1. Go back to Section 5
2. Change the `WORKSHOP_TOPIC` variable
3. Re-run the crew

**Example topics**:
- "RAG Systems with IBM watsonx.ai"
- "Prompt Engineering Best Practices"
- "AI Safety and Governance in Enterprise"

---

### Exercise 2: Add a New Agent (Medium)

**Goal**: Add a "Fact Checker" agent to verify the accuracy of content.

**Instructions**:
1. Create a new Agent with role "Fact Checker"
2. Create a Task that reviews the final article
3. Add both to the crew
4. Run the updated workflow

**Hints**:
- The fact checker should review the editor's output
- Use `context=[editing_task]` in the new task
- Make the agent verbose so you can see its reasoning

---

### Exercise 3: Custom Use Case (Advanced)

**Goal**: Build a multi-agent system for your own use case.

**Ideas**:
- Code review system (reviewer, tester, documenter)
- Customer support automation (triager, responder, escalator)
- Content moderation pipeline (scanner, reviewer, decision maker)
- Data pipeline validator (schema checker, quality analyzer, reporter)

**Challenge**: Use at least 3 agents with clear dependencies between tasks.

---

### Exercise 4: Add Model Parameters (Medium)

**Goal**: Experiment with different model parameters for different agents.

**Try**:
- Higher temperature (0.7-0.9) for creative agents
- Lower temperature (0.1-0.3) for analytical agents
- Different max_tokens for different tasks

**Question**: How does this affect the output quality?

---

### Exercise 5: Error Handling (Advanced)

**Goal**: Add proper error handling to the custom LLM wrapper.

**Tasks**:
1. Add try-except blocks in the `call()` method
2. Handle rate limiting gracefully
3. Add logging for debugging
4. Implement retry logic for transient failures

Use the code cell below for your solutions:

In [ ]:
# Your exercise code here
# Try the exercises above!

print("🎓 Exercise workspace ready!")
print("Copy and modify code from previous sections to complete the exercises.")

---

## 🎉 Conclusion & Next Steps

### What You've Learned

Congratulations! You've successfully:

✅ Connected to IBM watsonx.ai from Python  
✅ Built a custom LLM wrapper for CrewAI  
✅ Created specialized AI agents  
✅ Designed multi-agent workflows  
✅ Implemented real-world use cases  

---

### Key Takeaways

1. **Agent Design Matters**: Clear roles, goals, and backstories improve performance
2. **Task Dependencies**: Structured workflows produce better results
3. **Specialization Wins**: Multiple focused agents outperform single general-purpose ones
4. **watsonx.ai + CrewAI**: Powerful combination for enterprise AI workflows

---

### Next Steps & Extensions

#### 🔧 Technical Extensions

1. **Add Tool Integration**
   - Web search capabilities
   - File I/O operations
   - Database connections
   - API integrations

2. **Implement RAG (Retrieval Augmented Generation)**
   - Connect to vector databases (Milvus, Chroma)
   - Index your documentation
   - Enable semantic search

3. **Advanced Orchestration**
   - Hierarchical crews (manager agents)
   - Parallel task execution
   - Conditional workflows
   - Dynamic agent selection

4. **Production Features**
   - Add comprehensive logging
   - Implement monitoring and observability
   - Build error recovery mechanisms
   - Add rate limiting and cost controls

#### 📚 Learning Resources

- [IBM watsonx.ai Documentation](https://www.ibm.com/docs/en/watsonx-as-a-service)
- [CrewAI Documentation](https://docs.crewai.com/)
- [LangChain IBM Integration](https://python.langchain.com/docs/integrations/llms/ibm_watsonx/)

#### 💡 Use Case Ideas

1. **Developer Productivity**
   - Code review automation
   - Documentation generation
   - Test case creation

2. **Business Operations**
   - Customer support automation
   - Market research analysis
   - Competitive intelligence

3. **Content Creation**
   - Technical writing
   - Marketing content
   - Training materials

4. **Data Processing**
   - ETL pipeline validation
   - Data quality assessment
   - Report generation

---

### 📞 Get Help & Share Feedback

- **Questions?** Open an issue on GitHub or reach out on IBM Community
- **Found a bug?** Please report it!
- **Built something cool?** Share it with the community!

---

### 🙏 Thank You!

Thank you for completing this workshop! We hope you found it valuable and that you're excited to build amazing multi-agent systems with IBM watsonx.ai and CrewAI.

**Happy Building! 🚀**

---

*This notebook was created for educational purposes. Please review IBM's usage policies and pricing before deploying to production.*

---

## 📎 Appendix: Quick Reference

### Common watsonx.ai Model IDs

```python
# Granite Models
"ibm/granite-3-8b-instruct"          # Efficient instruction following
"ibm/granite-3-2b-instruct"          # Ultra-efficient for simple tasks
"ibm/granite-20b-multilingual"       # Multilingual support

# Other Models (availability varies by region)
"meta-llama/llama-3-70b-instruct"    # Large, capable model
"mistralai/mixtral-8x7b-instruct"    # Mixture of experts
```

### Useful Code Snippets

#### Create a Simple Agent
```python
agent = Agent(
    role="Your Role",
    goal="Your Goal",
    backstory="Your Backstory",
    llm=watsonx_llm,
    verbose=True
)
```

#### Create a Task
```python
task = Task(
    description="What to do",
    expected_output="What you want",
    agent=your_agent,
    context=[previous_task]  # Optional
)
```

#### Run a Crew
```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential,
    verbose=True
)
result = crew.kickoff()
```

### Troubleshooting

| Issue | Solution |
|-------|----------|
| Authentication error | Check API key and URL |
| Model not found | Verify model ID and region |
| Rate limiting | Add delays or reduce concurrent requests |
| Timeout | Reduce max_tokens or simplify prompts |
| Empty response | Check model parameters and prompt |

---